#### One To One

_One To One_ is essentially a `bidirectional relationship` with a __scalar attribute on both sides__. Within the ORM, _"one-to-one"_ is considered as a convention where the ORM expects that __only one related row will exist for any parent row__.

The _"one-to-one"_ convention is achieved by _applying a value_ of `False` to the `relationship.uselist` parameter of the `relationship()` construct, or in some cases the `backref()` construct, applying it on the _"one-to-many"_ or _"collection"_ side of a relationship.

In the example below we present a _bidirectional relationship_ that includes both `one-to-many (Parent.children)` and a `many-to-one (Child.parent)` relationships.

In [1]:
from sqlalchemy import Column, ForeignKey, Integer
from sqlalchemy.orm import backref, declarative_base, relationship

In [2]:
Base = declarative_base()

In [3]:
class Parent(Base):
    __tablename__ = "parent_table"
    id = Column(Integer, primary_key=True)
    # one-to-many collection
    children = relationship("Child", back_populates="parent")

In [4]:
class Child(Base):
    __tablename__ = "child_table"
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey("parent_table.id"))
    # many-to-one scalar
    parent = relationship("Parent", back_populates="children")

Above, `Parent.children` is the _"one-to-many"_ side __referring to a collection__, and `Child.parent` is the _"many-to-one"_ side __referring to a single object__. To convert this to _"one-to-one"_, the _"one-to-many"_ or _"collection"_ side is __converted into a scalar relationship__ using the `uselist=False` flag, renaming `Parent.children` to `Parent.child` for clarity.

In [5]:
class ParentModel(Base):
    __tablename__ = "parent"
    id = Column(Integer, primary_key=True)
    # previously one-to-many Parent.children is now one-to-one Parent.child
    child = relationship("ChildModel", back_populates="parent", uselist=False)

In [6]:
class ChildModel(Base):
    __tablename__ = "child"
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey("parent.id"))
    # many-to-one side remains, see tip below
    parent = relationship("ParentModel", back_populates="child")

Above, when we load a _Parent_ object, the _Parent.child_ attribute will __refer to a single Child object rather than a collection__. If we _replace_ the value of _Parent.child_ with a _new Child object_, the ORM's unit of work process will replace the previous Child row with the new one, setting the previous *child.parent_id* column to `NULL` __by default unless there are specific `cascade behaviors` set up__.

> ##### Note
>
> As mentioned previously, the ORM considers the _"one-to-one"_ pattern as a convention, where it _makes the assumption_ that when it loads the _Parent.child_ attribute on a _Parent_ object, it will __get only one row back__. If _more than one row_ is returned, the ORM will __emit a warning__.
>
> However, the _Child.parent_ side of the above relationship remains as a _"many-to-one"_ relationship and is __unchanged__, and there is __no intrinsic system__ within the ORM itself that _prevents more than one Child object to be created against the same Parent during persistence_. Instead, techniques such as `unique constraints` may be used in the actual database schema to _enforce_ this arrangement, where a _unique constraint_ on the *Child.parent_id* column would __ensure__ that _only one Child row may refer to a particular Parent row_ at a time.

In the case where the `relationship.backref` parameter is used to define the _"one-to-many"_ side, this can be converted to the _"one-to-one"_ convention using the `backref()` function which allows the relationship generated by the `relationship.backref` parameter to __receive custom parameters__, in this case the _uselist_ parameter.

In [7]:
class ParentTable(Base):
    __tablename__ = "table_parent"
    id = Column(Integer, primary_key=True)

In [8]:
class ChildTable(Base):
    __tablename__ = "table_child"
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey("table_parent.id"))
    parent = relationship("ParentTable", backref=backref("child", uselist=False))